In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("~/mlops_project/data/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

In [37]:
df.groupby("Sex")["Age"].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

In [38]:
df['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [39]:
fill_values = {'female': 28, 'male': 31}
df['Age'].fillna(df['Sex'].map(fill_values), inplace=True)
df['Embarked'].fillna('S', inplace=True)

In [50]:
df_final = pd.get_dummies(df, drop_first=True, dtype=int)

In [61]:
X = df_final.drop('Survived', axis=1)
y = df_final['Survived']

In [66]:
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import optuna
from optuna.trial import Trial

In [ ]:
def optimize_rfc(trial: Trial,
                 data: tuple,
                 evaluate_metric: str = 'accuracy',
                 seed: int = 42):
    """Оптимизация гиперпараметров RandomForestClassifier.
    
    Args:
        trial: Объект испытания Optuna
        data: Кортеж (X, y) с данными
        evaluate_metric: Метрика для оценки (по умолчанию 'accuracy')
        seed: Random state для воспроизводимости
        
    Returns:
        Среднее значение метрики по кросс-валидации
    """
    
    data_x, data_y = data
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy'])
    }

    model = RandomForestClassifier(**params, random_state=seed)
    score = cross_val_score(model, data_x, data_y, cv=5, scoring=evaluate_metric).mean()

    return score

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(
    lambda trial: optimize_rfc(trial, data=(X, y)),
    n_trials=100,
    show_progress_bar=True
)